In [ ]:
# import libraries
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from imblearn.over_sampling import RandomOverSampler



print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Cargar datos de entrenamiento y validación
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
valid_data = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])



In [ ]:
# Convertir etiquetas a valores numericos
train_data['label'] = train_data['label'].map({'ham': 0, 'spam': 1})
valid_data['label'] = valid_data['label'].map({'ham': 0, 'spam': 1})

In [ ]:
# Tokenización y secuenciación
vocab_size = 10000
max_length = 120

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(train_data['message'])


In [ ]:
# Convertir textos a secuencias
train_sequences = tokenizer.texts_to_sequences(train_data['message'])
valid_sequences = tokenizer.texts_to_sequences(valid_data['message'])

In [ ]:
# Padding de secuencias
train_padded = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, padding='post', maxlen=100)
valid_padded = tf.keras.preprocessing.sequence.pad_sequences(valid_sequences, padding='post', maxlen=100)

In [ ]:
# Convertir etiquetas a numpy arrays
train_labels = np.array(train_data['label'])
valid_labels = np.array(valid_data['label'])

In [ ]:
# Sobremuestreo
ros = RandomOverSampler(random_state=42)
train_padded_resampled, train_labels_resampled = ros.fit_resample(train_padded, train_labels)

In [ ]:
# Definir el modelo
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=16, input_length=max_length),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compilar el modelo
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])



In [ ]:
# Entrenar el modelo
history = model.fit(train_padded_resampled, train_labels_resampled, epochs=30, validation_data=(valid_padded, valid_labels), verbose=2, batch_size=64)


In [ ]:
def predict_message(pred_text):

    seq = tokenizer.texts_to_sequences([pred_text])
    padded = tf.keras.preprocessing.sequence.pad_sequences(seq, padding='post', maxlen=max_length)

    prediction = model.predict(padded)[0][0]

    label = 'spam' if prediction > 0.5 else 'ham'

    return [float(prediction), label]

pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)



In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
